In [1]:
### REPORT BUILDER ####
# !pip install feedparser
#!pip install spacy
#!pip install transformers
#!python -m spacy download en_core_web_sm


In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import requests
import spacy
import feedparser


class PreprocessorModule:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")

    def generate_search_queries(self, topic):
            # Step 1: Input Processing
            topic = topic.lower() # convert the text to lowercase
            print(f"Step 1: Lowercase topic: {topic}")
            topic_tokens = word_tokenize(topic)

            # Step 2: Part-of-Speech (POS) Tagging
            topic_pos_tags = nltk.pos_tag(topic_tokens)
            print(f"Step 2: POS tags: {topic_pos_tags}")

            # Step 3: Named Entity Recognition (NER)
            # Implement a NER algorithm of your choice here to identify named entities in the topic

            # Step 4: Stopword Removal
            stop_words = set(stopwords.words('english'))
            topic_filtered_tokens = [word for word in topic_tokens if not word in stop_words]
            print(f"Step 4: Filtered tokens (stopwords removed): {topic_filtered_tokens}")

            # Step 5: Stemming
            ps = PorterStemmer()
            topic_stemmed_tokens = [ps.stem(word) for word in topic_filtered_tokens]
            print(f"Step 5: Stemmed tokens: {topic_stemmed_tokens}")

            # Step 6: Query Generation
            search_query = " ".join(topic_filtered_tokens)
            print(f"Step 6: Search query: {search_query}")

            # Step 7: Query Refinement
            for i in range(3):
                # Ask the user to provide feedback on the initial search results
                feedback = input("Please provide feedback on the initial search results: ")
                
                # If the user is satisfied with the results, return the search queries
                if feedback == "satisfied":
                    return [search_query]

            # If the user is still not satisfied after three rounds of refinement, return the original search queries
            print(f"User not satisfied after 3 rounds. Returning original search query: {search_query}")
            return [search_query]


In [2]:
import requests
from scholarly import scholarly

class DataRetrievalModule:

    def search_google_scholar(self, query):
        # Add code to search using the Google Scholar API
        search_results = []
        try:
            print(f"Searching Google Scholar with query: {query}")
            search_query = scholarly.search_pubs(query.encode("utf-8"))
            print(f"Google Scholar search query: {search_query}")
            for result in search_query:
                search_results.append({
                    "title": result["bib"]["title"],
                    "url": result["pub_url"],
                    "authors": result["bib"].get("author", ""),
                    "abstract": result["bib"].get("abstract", ""),
                    "publication_date": result["bib"].get("pub_year", ""),
                    "source": "Google Scholar"
                })
        except Exception as e:
            print(f"Error while searching Google Scholar: {e}")
        print(f"Google Scholar search results: {search_results}")
        return search_results

    def search_arxiv(self, query):
        base_url = "http://export.arxiv.org/api/query"
        params = {
            "search_query": f"all:{query}",
            "start": 0,
            "max_results": 10,
            "sortBy": "relevance",
            "sortOrder": "descending"
        }
        print(f"Searching ArXiv with query: {query}")
        print(f"ArXiv search parameters: {params}")
        response = requests.get(base_url, params=params)
        feed = feedparser.parse(response.content)

        results = []
        for entry in feed.entries:
            results.append({
                "title": entry["title"],
                "url": entry["link"],
                "authors": [author["name"] for author in entry["authors"]],
                "publication_date": entry["published"],
                "source": "ArXiv"
            })
        print(f"ArXiv search results: {results}")
        return results

    def retrieve_data(self, query):
        results = []
        print(f"Retrieving data with query: {query}")
        results.extend(self.search_google_scholar(query))
        results.extend(self.search_arxiv(query))
        filtered_results = self.filter_and_sort(results)
        print(f"Filtered and sorted results: {filtered_results}")
        return filtered_results

    def filter_and_sort(self, results):
        # Filter out results without a publication date
        filtered_results = [r for r in results if r["publication_date"]]
        print(f"Filtering results without a publication date: {filtered_results}")

        # Calculate a quality score based on the number of authors
        for r in filtered_results:
            r["quality_score"] = len(r["authors"])
        print(f"Calculating quality score for each result: {filtered_results}")

        # Sort the results by the quality score, in descending order
        sorted_results = sorted(filtered_results, key=lambda x: x["quality_score"], reverse=True)
        print(f"Sorted results by quality score: {sorted_results}")

        return sorted_results


In [3]:
import spacy
from transformers import pipeline

class DataAnalysisModule:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.summarizer = pipeline("summarization")

    def preprocess(self, text):
        # Add code to preprocess the text, e.g., remove special characters or extra whitespace
        processed_text = text.strip()
        print(f"Preprocessed text: {processed_text}")
        return processed_text

    def extract_entities(self, text):
        # Add code to extract named entities using the NLP library
        doc = self.nlp(text)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        print(f"Extracted entities: {entities}")
        return entities

    def extract_keywords(self, text):
        # Add code to extract keywords and phrases from the text
        doc = self.nlp(text)
        keywords = [chunk.text for chunk in doc.noun_chunks]
        print(f"Extracted keywords: {keywords}")
        return keywords

    def summarize(self, text):
        # Add code to generate a summary using the Hugging Face Transformers library
        summary = self.summarizer(text, max_length=2000, min_length=500, do_sample=False)
        print(f"Generated summary: {summary[0]['summary_text']}")
        return summary[0]['summary_text']

    def paraphrase(self, text):
        # Add code to paraphrase the text using an appropriate NLP technique or model
        # Note: Paraphrasing is a complex task and requires a dedicated model.
        # For now, the function returns the input text.
        paraphrased_text = text
        print(f"Paraphrased text (not implemented): {paraphrased_text}")
        return paraphrased_text

    def analyze_data(self, data):
        processed_data = []
        for item in data:
            print(f"Analyzing item: {item['title']}")
            text = self.preprocess(item["content"])
            entities = self.extract_entities(text)
            keywords = self.extract_keywords(text)
            summary = self.summarize(text)
            paraphrased_content = self.paraphrase(summary)

            processed_data.append({
                "title": item["title"],
                "url": item["url"],
                "entities": entities,
                "keywords": keywords,
                "summary": summary,
                "paraphrased_content": paraphrased_content
            })

        print(f"Processed data: {processed_data}")
        return processed_data


In [4]:
from transformers import pipeline
import openai

class ReportGenerationModule:
    def __init__(self):
        self.api_key = "CokHFBZqdMK1VfgaaUbiT3BlbkFJRCTVNL9WTgbwuAMks9kF"
        openai.api_key = self.api_key

    def create_prompt(self, data_item):
        # Add code to create a prompt based on the analyzed data
        prompt = f"{data_item['title']} - {data_item['paraphrased_content']}"
        print(f"Created prompt: {prompt}")
        return prompt

    def generate_text(self, prompt):
        # Add code to generate text using the language model
        response = openai.Completion.create(
            engine="gpt-3.5-turbo",
            prompt=prompt,
            max_tokens=3500,
            n=1,
            stop=None,
            temperature=0.7,
        )

        generated_text = response.choices[0].text.strip()
        print(f"Generated text: {generated_text}")
        return generated_text

    def combine_sections(self, sections):
        # Add code to combine the generated text into a structured report
        report = "\n".join(sections)
        print(f"Combined sections: {report}")
        return report

    def add_citations(self, report, data_items):
        # Add code to add citations and references to the report
        citations = []
        for i, data_item in enumerate(data_items, start=1):
            citations.append(f"[{i}] {data_item['url']}")

        report_with_citations = f"{report}\n\nReferences:\n" + "\n".join(citations)
        print(f"Report with citations: {report_with_citations}")
        return report_with_citations

    def generate_report(self, analyzed_data):
        report_sections = []
        for data_item in analyzed_data:
            print(f"Generating report section for: {data_item['title']}")
            prompt = self.create_prompt(data_item)
            generated_text = self.generate_text(prompt)
            report_sections.append(generated_text)

        report = self.combine_sections(report_sections)
        report_with_citations = self.add_citations(report, analyzed_data)
        print(f"Generated report: {report_with_citations}")
        return report_with_citations


In [5]:
import language_tool_python
import nltk
nltk.download('stopwords')

class PostprocessorModule:
    def __init__(self):
        self.language_tool = language_tool_python.LanguageTool("en-US")

    def check_grammar_and_spelling(self, text):
        # Add code to check grammar and spelling using LanguageTool
        matches = self.language_tool.check(text)
        print(f"Grammar and spelling issues found: {matches}")
        return matches

    def review_report(self, report):
        # Add code to review the report, including grammar, spelling, and plagiarism checks
        # This example focuses on grammar and spelling checks using LanguageTool
        grammar_and_spelling_issues = self.check_grammar_and_spelling(report)
        review_results = {
            "grammar_and_spelling": grammar_and_spelling_issues
        }
        print(f"Review results: {review_results}")
        return review_results

    def generate_final_report(self, report, review_results):
        # Add code to generate the final report by addressing any issues found in the review process
        final_report = report
        for issue in review_results["grammar_and_spelling"]:
            try:
                suggested_correction = issue.replacements[0]
                final_report = final_report.replace(issue.matched_text, suggested_correction)
            except IndexError:
                continue

        print(f"Final report: {final_report}")
        return final_report

    def process_report(self, report):
        print("Starting report postprocessing...")
        review_results = self.review_report(report)
        final_report = self.generate_final_report(report, review_results)
        print("Finished report postprocessing.")
        return final_report


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladbordei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
class ReportGenerator:
    def __init__(self):
        self.preprocessor = PreprocessorModule()
        self.data_retrieval = DataRetrievalModule()
        self.data_analysis = DataAnalysisModule()
        self.report_generation = ReportGenerationModule()
        self.postprocessor = PostprocessorModule()

    def generate_search_queries(self, topic):
        search_queries = self.preprocessor.generate_search_queries(topic)
        return search_queries

    def retrieve_data(self, search_queries):
        retrieved_data = self.data_retrieval.retrieve_data(search_queries)
        return retrieved_data

    def analyze_data(self, retrieved_data):
        analyzed_data = self.data_analysis.analyze_data(retrieved_data)
        return analyzed_data

    def generate_initial_report(self, analyzed_data):
        initial_report = self.report_generation.generate_report(analyzed_data)
        return initial_report

    def process_report(self, initial_report):
        final_report = self.postprocessor.process_report(initial_report)
        return final_report

    def generate_report(self, topic, additional_requirements=None):
        search_queries = self.generate_search_queries(topic)
        retrieved_data = self.retrieve_data(search_queries)
        analyzed_data = self.analyze_data(retrieved_data)
        initial_report = self.generate_initial_report(analyzed_data)
        final_report = self.process_report(initial_report)

        return final_report


report_generator = ReportGenerator()

topic = "Natural Language Processing"
search_queries = report_generator.generate_search_queries(topic)
retrieved_data = report_generator.retrieve_data(search_queries)
analyzed_data = report_generator.analyze_data(retrieved_data)
initial_report = report_generator.generate_initial_report(analyzed_data)
final_report = report_generator.process_report(initial_report)

print(final_report)

In [8]:
# Example usage:
report_generator = ReportGenerator()
topic = "Integrating Battery Storage Systems with Renewable Energy Sources on Site"

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [9]:
search_queries = report_generator.generate_search_queries(topic)

Step 1: Lowercase topic: integrating battery storage systems with renewable energy sources on site
Step 2: POS tags: [('integrating', 'VBG'), ('battery', 'NN'), ('storage', 'NN'), ('systems', 'NNS'), ('with', 'IN'), ('renewable', 'JJ'), ('energy', 'NN'), ('sources', 'NNS'), ('on', 'IN'), ('site', 'NN')]
Step 4: Filtered tokens (stopwords removed): ['integrating', 'battery', 'storage', 'systems', 'renewable', 'energy', 'sources', 'site']
Step 5: Stemmed tokens: ['integr', 'batteri', 'storag', 'system', 'renew', 'energi', 'sourc', 'site']
Step 6: Search query: integrating battery storage systems renewable energy sources site
